In [54]:
import numpy as np
import scipy.io as sio
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
import matplotlib.pyplot as plt
import os

# Enable GPU if available
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)

# Function to load MAT files from subfolders
def load_mat_files_by_roof_type(main_folder_path):
    data_by_roof = {}
    subfolders = [item for item in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, item))]
    for subfolder in subfolders:
        subfolder_path = os.path.join(main_folder_path, subfolder)
        data_by_roof[subfolder] = {}
        for file in os.listdir(subfolder_path):
            if file.endswith(".mat"):
                mat_data = sio.loadmat(os.path.join(subfolder_path, file))
                data_by_roof[subfolder][file] = mat_data
    return data_by_roof

# Specify the folder containing MAT files
folder_path = "C:/Users/rrsuj/Downloads/capstone/data"
data_dict_by_roof = load_mat_files_by_roof_type(folder_path)

# Extract pressure coefficient values and corresponding metadata by roof type
X_train_by_roof = {}
Y_train_by_roof = {}

for roof_type, mat_files in data_dict_by_roof.items():
    X_train_by_roof[roof_type] = []
    Y_train_by_roof[roof_type] = []
    for filename, mat_data in mat_files.items():
        if 'Cp' in mat_data:
            Cp = mat_data['Cp']
            try:
                wind_angle_str = filename.split("_deg")[1].replace(".mat", "")
                wind_angle = float(wind_angle_str)
                X_train_by_roof[roof_type].append(np.full_like(Cp, wind_angle))
                Y_train_by_roof[roof_type].append(Cp.flatten())
            except (IndexError, ValueError):
                print(f"Warning: Could not extract wind angle from filename: {filename} in {roof_type}")

# Convert to NumPy arrays
for roof_type in X_train_by_roof:
    if X_train_by_roof[roof_type]:
        X_train_by_roof[roof_type] = np.array(X_train_by_roof[roof_type]).reshape(-1, 1)
        Y_train_by_roof[roof_type] = np.array(Y_train_by_roof[roof_type]).reshape(-1, 1)
    else:
        X_train_by_roof[roof_type] = np.array([]).reshape(0, 1)
        Y_train_by_roof[roof_type] = np.array([]).reshape(0, 1)

print("Training Data Shapes by Roof Type:")
for roof_type in X_train_by_roof:
    if X_train_by_roof[roof_type].size > 0:
        print(f"{roof_type}: X_train shape = {X_train_by_roof[roof_type].shape}, Y_train shape = {Y_train_by_roof[roof_type].shape}")
    else:
        print(f"Warning: No training data found for {roof_type}")

# Physics-Informed Loss (Bernoulli-based correction)
def physics_loss(y_true, y_pred):
    rho = 1.225  # Air density (kg/m³)
    V_ref = 10  # Reference wind speed (m/s)

    # Bernoulli-based pressure coefficient correction
    Cp_physics = 2 * (y_pred / (rho * V_ref**2))

    # MSE loss between data and physics-corrected prediction
    return tf.reduce_mean(tf.square(y_true - Cp_physics))

# Define Neural Network Architecture
def build_pinn():
    inputs = Input(shape=(1,))
    x = Dense(64, activation="tanh")(inputs)
    x = Dense(64, activation="tanh")(x)
    x = Dense(64, activation="tanh")(x)
    outputs = Dense(1, activation="linear")(x)  # Output mean pressure coefficient

    model = Model(inputs, outputs)
    return model

# Train the PINN model for each roof type
history_by_roof = {}
pinn_models_by_roof = {}

for roof_type in X_train_by_roof:
    if X_train_by_roof[roof_type].size > 0:
        print(f"\nTraining PINN for {roof_type}...")
        pinn = build_pinn()
        pinn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=physics_loss)
        pinn.summary()
        history = pinn.fit(X_train_by_roof[roof_type], Y_train_by_roof[roof_type], epochs=500, batch_size=32, verbose=1)
        history_by_roof[roof_type] = history
        pinn_models_by_roof[roof_type] = pinn

        # Plot Training Loss
        plt.plot(history.history['loss'], label="Training Loss")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend()
        plt.title(f"Training Loss of PINN Model for {roof_type}")
        plt.show()
    else:
        print(f"Skipping training for {roof_type} due to lack of data.")

# Test wind angles (replace empty array with test values)
test_angles = np.array([[0], [30], [60], [90]])

# Predict pressure coefficients for each roof type
predicted_Cp_by_roof = {}
for roof_type, model in pinn_models_by_roof.items():
    predicted_Cp = model.predict(test_angles)
    predicted_Cp_by_roof[roof_type] = predicted_Cp

    # Display Predictions
    print(f"\nPredictions for {roof_type}:")
    for i, angle in enumerate(test_angles.flatten()):
        print(f"Predicted Mean Pressure Coefficient at {angle}°: {predicted_Cp[i][0]:.4f}")


Training Data Shapes by Roof Type:
Skipping training for roof type a due to lack of data.
Skipping training for roof type b due to lack of data.
Skipping training for roof type c due to lack of data.
Skipping training for roof type o due to lack of data.
